# Sentiment Analysis with Deep Learning

# Phase 2- Modelling

This notebook consists the functions and code for modelling.  

### CHRISP-DM phases

Modelling and Evaluation phases for CRISP-DM can be found in this noteboook.

#### 4.Modeling
Modeling techniques are selected and applied. 

#### 5.Evaluation
Once one or more models have been built that appear to have high quality based on whichever loss functions have been selected, these need to be tested to ensure they generalize against unseen data and that all key business issues have been sufficiently considered.  The end result is the selection of the champion model(s).

### Table of Contents

- 1.Import Libraries
- 2.Define Functions 
- 3.Modeling With Neural Networks  
- 4.Tuning the Best Model

## 1. Import Libraries

In [ ]:
import pandas as pd
import gc
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns
import nltk
from nltk import word_tokenize, FreqDist
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
np.random.seed(0)
import pickle

In [ ]:
from keras.models import Model, Sequential, Input
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPool1D, GlobalAveragePooling1D, Dropout, concatenate, Layer, InputSpec, CuDNNLSTM, SpatialDropout1D, Activation, LSTM
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints, optimizers, layers
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, SpatialDropout1D, Activation
from keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization
from keras.optimizers import Adam, SGD
import pickle

## 2. Functions 

In [ ]:
# number of unique words we want to use (or: number of rows in incoming embedding vector)
max_features = 8192
# max number of words in a comment to use (or: number of columns in incoming embedding vector)
max_len = 128
# dimension of the embedding variable (or: number of rows in output of embedding vector)
embedding_dims = 64

In [ ]:
def load_smalldata ( ):
    # Loading partial train test files , tokenized, sequenzed and padded
    pickle_in = open("data/vectors_small/X_train2_file.pickle","rb")
    X_train2 = pickle.load(pickle_in)

    pickle_in = open("data/vectors_small/X_test2_file.pickle","rb")
    X_test2 = pickle.load(pickle_in)

    pickle_in = open("data/vectors_small/y_train2_file.pickle","rb")
    y_train2 = pickle.load(pickle_in)

    pickle_in = open("data/vectors_small/y_test2_file.pickle","rb")
    y_test2 = pickle.load(pickle_in)
    
    return X_train2, X_test2, y_train2, y_test2


def load_tokenizer():
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer=pickle.load(handle)
        return tokenizer
    
def create_model ( hidden_layers, 
                  loss='binary_crossentropy',
                  optimizer=Adam(0.01),
                  metrics=['accuracy'],
                  embedding_matrix=None,
                  max_len=max_len,
                  embedding_dims=embedding_dims,
                  max_features=max_features,
                  glove=False,
                 ):
 
    # check if embedding matrix has assigned which means the model uses glove embeddings 
    if glove==False:
        emb_layer=Embedding(input_dim=max_features, input_length=max_len,
                        output_dim=embedding_dims)
    else:
        
        emb_layer=Embedding(input_dim =embedding_matrix.shape[0], input_length=max_len,
                          output_dim=embedding_matrix.shape[1], 
                          weights=[embedding_matrix], trainable=False)
    
    # instantiate Sequential model
    model = Sequential()
 
    # add embedding layer with defined parameters
    model.add(emb_layer)
   
    # add hidden layers available in hidden_layers list
    for layer in hidden_layers:
        model.add(layer)
    
    # add pooling layer 
    model.add(GlobalMaxPool1D())

    # set the dropout layer to drop out 50% of the nodes
    model.add(Dropout(0.5))

    # add dense layer to produce an output dimension of 50 and using relu activation
    model.add(Dense(50, activation='relu'))

    # finally add a dense layer
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=metrics)
    
    model.summary() 
    return model


def run_model(model, model_name, results, epochs, batch_size=32):
    hist = model.fit(X_train2, y_train2, 
                     batch_size=batch_size, 
                     epochs=epochs, 
                     validation_split=0.1)

    test_loss, test_auc = model.evaluate(X_test2, y_test2, batch_size=32)
    print("-------------------------------------------")
    print("")
    print("")
    print(model_name + ' Test Loss:    ', test_loss)
    print(model_name + ' Test Accuracy:', test_auc)
    print("")
    print("")
    print("")
    
    
    #Pass the results as key value pairs to append() function 
    row=[]
    row =[model_name , hist.history['accuracy'][-1],
          hist.history['val_accuracy'][-1],test_auc, test_loss] 

    save_model(model, model_name)
    return row



def save_model (model, model_name):
    path="models/"+model_name + ".h5"
    model.save(path)

def save_results (results, row):
    #Pass the results as key value pairs to append() function 
    results = results.append({'model' : row[0] , 
                        'train_acc' : row[1], 
                        'val_acc':row[2],
                        'test_acc': row[3]
                              
                                    } , ignore_index=True)
    return results


### Loading Glove Dictionary
def load_glove (path):
    # load the glove840B embedding
    embeddings_index = dict()
    f = open(path)

    for line in f:
        # Note: use split(' ') instead of split() if you get an error
        values = line.split(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Loaded %s word vectors.' % len(embeddings_index))

    # create a weight matrix
    embedding_matrix = np.zeros((len(tokenizer.word_index)+1, 300))

    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


## 3. Modeling With Neural Networks                       

I will first try 4 different models to see which one gives the best result.

#### Set parameters for modeling

#### Create a dataframe to store the accuarecy results

In [ ]:
results=pd.DataFrame(columns=["model", "train_acc","val_acc" ,"test_acc"])

In [ ]:
#load small dataset to train and test the models
X_train2, X_test2, y_train2, y_test2=load_smalldata()
#load tokenizer
tokenizer=load_tokenizer()

#### Define parameters for each model such as hidden layers and glove

In [ ]:
dnn_params={'hidden_layers':[],
            'glove' : False,
            'embedding_matrix':None
           }


cnn_params = {
        'hidden_layers':[SpatialDropout1D(0.5),
                   Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'),
                   BatchNormalization()],
        'glove' : False,
        'embedding_matrix':None}


rnn_params ={ 
     'hidden_layers':[SpatialDropout1D(0.5),
                  Bidirectional(LSTM(25, 
                  return_sequences=True))],
     'glove' : False,
     'embedding_matrix':None}

#load glove embedding vectors from txt file
embedding_matrix=load_glove ("glove.6B.300d.txt")

cnn_glove_params ={
    'hidden_layers':[SpatialDropout1D(0.5),
                   Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'),
                   BatchNormalization()],
    'glove' : True,
    
    'embedding_matrix':embedding_matrix
    }

rnn_glove_params ={
    'hidden_layers':[SpatialDropout1D(0.5),
                  Bidirectional(LSTM(25, 
                  return_sequences=True))],
    'glove' : True,
    'embedding_matrix':embedding_matrix
    }


In [19]:
# Create lists for model names and hiden layer
names=["dnn","cnn", "rnn", "cnn_glove", "rnn_glove"]
params=[dnn_params, cnn_params, rnn_params, 
        cnn_glove_params, rnn_glove_params]
epochs=1

#### For loop for modelling

Run a for loop to create the model, compile, fit and save results and the model itsef

In [ ]:
#set epochs to 3
#Run a for loop to create the model, compile, fit and save results 
#and save the model to models folder
for name, param in zip(names, params):
    print("Model Name :", name)
    print("======================")
    model=create_model(hidden_layers=param['hidden_layers'], 
                   glove=param['glove'],
                   embedding_matrix=param['embedding_matrix']
                    )
    row=run_model(model, name, results, epochs)
    results=save_results(results, row)


Model Name : dnn
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 128, 64)           524288    
_________________________________________________________________
global_max_pooling1d_8 (Glob (None, 64)                0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 50)                3250      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 51        
Total params: 527,589
Trainable params: 527,589
Non-trainable params: 0
_________________________________________________________________
Train on 135000 samples, validate on 15000 samples
Epoch 1/1
30000/30000 [=======================

/Users/kaan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 135000 samples, validate on 15000 samples
Epoch 1/1
30000/30000 [==============================] - 13s 434us/step
-------------------------------------------


cnn Test Loss:     0.6933107460975647
cnn Test Accuracy: 0.5032333135604858



Model Name : rnn
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 128, 64)           524288    
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 128, 64)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128, 50)           18000     
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 50)                0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 50)             

/Users/kaan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 135000 samples, validate on 15000 samples
Epoch 1/1
30000/30000 [==============================] - 40s 1ms/step
-------------------------------------------


rnn Test Loss:     0.48579550501505536
rnn Test Accuracy: 0.7744666934013367



Model Name : cnn_glove
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 128, 300)          263894100 
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 128, 300)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 128, 100)          120100    
_________________________________________________________________
batch_normalization_3 (Batch (None, 128, 100)          400       
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 100)       

## Checking results and picking the model for tuning


In [ ]:
results

Based on this results CNN is the best performing model so far. From this forward, I will tune only this model and try to impove its performance. 

## 4. Tuning the Best Model

## Iteration 1:  CNN with SGD optimizer

I will use a different optimizer for the same model structure. I will use SGD.  

In [ ]:
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.5, nesterov=True)

In [ ]:
# iteration 1 = with sgd optimizer
model=create_model(hidden_layers=[SpatialDropout1D(0.5),
                        Conv1D(filters=100,kernel_size=4, padding='same', activation='relu'),
                        BatchNormalization()],
                      glove = False,
                      embedding_matrix=None,
                      optimizer = sgd)
row=run_model(model, "cnn_glove_sgd", results, epochs)

results=save_results(results, row)
 

SGD optimizer did not perform better than "adam". So lets stick with "adam" optimizer.  

## Iteration 2:  CNN with 2 convolution layers


In [ ]:

model=create_model(hidden_layers=[SpatialDropout1D(0.5),
                        Conv1D(filters=100,kernel_size=4, padding='same', activation='relu'),
                        BatchNormalization(),
                        Conv1D(filters=100,kernel_size=4, padding='same', activation='relu'),
                        BatchNormalization()],
                        glove =False,
                        embedding_matrix=None,
                      )
epochs=3

row=run_model(model, "cnn_2cnv", results, epochs)

results=save_results(results, row)
 

With the second convolution layer cnn model performed slighly better. 



## Iteration 3:  CNN with more epochs

In [ ]:

model=create_model(hidden_layers=[SpatialDropout1D(0.5),
                        Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'),
                        BatchNormalization()],
                      glove = False,
                      embedding_matrix=None)
epochs=8
row=run_model(model, "cnn_glove_8epochs", results, epochs)
results=save_results(results, row)
 

In [ ]:
results

## Conclusions

I have trained 5 models as base models and choose the best performing one to tune. Since deep learning models need a lot of computational power and  memory, I have used small sample from the dataset to train these model . Those 5 are :  

- Dense Neural Network
- Convolutional Neural Network
- Recurring Neural Network
- CNN with Glove Embedding
- RNN with Glove Embedding

Glove embedding is a pre-trained dictionary that would help for my model . But in the end CNN model was the best performing one. 

I have tuned convolution neural network. My laptop’s CPU could not handle the  training with full dataset. So I trained the model on Kaggle. Kaggle is a website that provides a platform for us to deal with massive dataset and heavy computations for machine learning. 



## Future Work

I would compare the different word scoring methods for tuning my model. I used ‘binary’’  but there are others such as “count”, “tfidf”, “freq” Those methods are used to select the most impactful words that would be used for model.


***Using different vectorizers is in progress.***


## Iteration 4:  CNN with different vectorizations

In [ ]:
model=create_model(hidden_layers=[SpatialDropout1D(0.5),
                        Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'),
                        BatchNormalization()],
                      glove = False,
                      embedding_matrix=None)
epochs=1

In [ ]:
pickle_in = open("data/tokens/test_file.pickle","rb")
test = pickle.load(pickle_in)

pickle_in = open("data/tokens/train_file.pickle","rb")
train = pickle.load(pickle_in)


In [ ]:
import random
train2=train.sample(n=80000, random_state=1)
train2=train2.reset_index()
test2=test.sample(n=20000, random_state=1)
test2=test2.reset_index()

In [ ]:
def label_split (train, test):
    X_train=train.comment
    X_test=test.comment
    y_train=train.label
    y_test=test.label
    return (X_train, X_test, y_train, y_test)

In [ ]:
#split label and comment columns in the data
X_train2, X_test2, y_train2, y_test2 = label_split(train2, test2)


#### Initiate vectorizers

In [ ]:
tfidfvec = TfidfVectorizer(stop_words='english', tokenizer=tokenizer)
tfidfvec2 = TfidfVectorizer(stop_words='english', tokenizer=tokenizer, ngram_range=(1,2))
tfidfvec3 = TfidfVectorizer(stop_words='english', tokenizer=tokenizer, ngram_range=(1,3))
countvec = CountVectorizer(stop_words='english', tokenizer=tokenizer)
countvec2 = CountVectorizer(stop_words='english', tokenizer=tokenizer, ngram_range=(1,2))
countvec3 = CountVectorizer(stop_words='english', tokenizer=tokenizer, ngram_range=(1,3))

In [ ]:
names=["cnn_tfidfvec","cnn_tfidfvec2", "cnn_tfidfvec3", "cnn_countvec", "cnn_countvec2", "cnn_countvec3"]
vectorizers=[tfidfvec,tfidfvec2,tfidfvec3,countvec,countvec2,countvec3]

In [ ]:

#def model_performance(vectorizer, train_data, test_data, y_test):
accuracy_df = []
for name, vectorizer in zip(names, vectorizers):

    X_train2 = vectorizer.fit_transform(train2)
    X_test2 = vectorizer.transform(test2)

    print("Model Name :", name)
    print("======================")

    row=run_model(model, name, results, epochs)
    results=save_results(results, row)
